## Importing libs and setting plotting parameters

In [1]:
import pandas as pd
import pickle
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from datetime import timedelta, datetime

### Null values in `subsegmento` are filled with `N/A`, `valor_emprestado` is set as an attribute with type `int`, and the columns `desembolso` and `vencimento` are set to datetime type. Since now id values are unique, they are set as the index of the DataFrame.

In [2]:
with open('../data/processed/train_timeseries_attributes.p', 'rb') as handle:
    dict_loan_timeseries = pickle.load(handle)

df_loans = pd.read_csv('../data/processed/train_constant_attributes.csv')

df_loans["desembolso"] = pd.to_datetime(df_loans["desembolso"], format="%Y-%m-%d")
df_loans["vencimento"] = pd.to_datetime(df_loans["vencimento"], format="%Y-%m-%d")
df_loans['subsegmento'].fillna('N/A', inplace=True)

df_loans.index = df_loans['id']
df_loans.drop(columns=['id'], inplace=True)
df_loans.head()

,desembolso,vencimento,valor_emprestado,pgto_diario_esperado,subsegmento,y
id,,,,,,
6,2019-03-25,2019-12-26,7100.0,33.8852,N/A,0
8,2019-03-25,2019-12-26,6900.0,32.9307,N/A,0
9,2019-03-25,2019-12-26,42500.0,175.5301,N/A,0
11,2019-03-25,2019-12-26,4200.0,19.2553,Alimentação Rápida,0
12,2019-03-06,2019-12-05,7700.0,34.0856,Bares e Restaurantes,0


In [3]:
# unique indexes
unique_index = df_loans.index

### Building App

In [4]:
app = JupyterDash(__name__, assets_folder='../assets')

app.layout = html.Div([
    html.Div(id='header-info', className='header-info'),
    dcc.Dropdown(
        id='company-id', clearable=False,
        value=unique_index[0], options=[{'label': f'Loan ID: {index}', 'value': index}
                                        for index in unique_index], className='dropdown-select-id'
    ),
    html.Div([
        dcc.Graph(id='graph-debt', className='graph-id'),
        dcc.Graph(id='graph-transaction-payment', className='graph-id')
    ], className='container-graph-id'),

    html.Div(id='table-const-id', className='table-hidden'),
    html.Div(id='table-timeseries-id', className='table-hidden'),
])


@app.callback(
    Output('table-const-id', 'children'),
    Output('table-timeseries-id', 'children'),
    Input('company-id', 'value')
)
def update_table(loan_id):
    id_tmp_db_const = df_loans.loc[loan_id]
    id_tmp_db_timeseries = dict_loan_timeseries[loan_id]

    return id_tmp_db_const.to_json(date_format='iso', orient='split'), id_tmp_db_timeseries.to_json(date_format='iso', orient='split')


def create_time_series(df, title):
    fig = px.line(df, x='dia', y='value', color='variable', title=title)
    fig.update_xaxes(showgrid=False)
    fig.update_yaxes(title='Reais (R$)', visible=True, showticklabels=True)
    fig.update_xaxes(title='', visible=True, showticklabels=True)
    fig.update_layout(legend_orientation='h')
    fig.update_layout(legend_title_text='')
    fig.update_layout(height=300, margin={'l': 30, 'b': 30, 'r': 30, 't': 30})

    day_first, _ = df['dia'].iloc[0].split('T')
    day_ninety = datetime.strptime(day_first, '%Y-%m-%d') + timedelta(days=90)

    fig.add_vline(x=day_ninety.timestamp() * 1000, line_width=3, line_dash="dash",
                  line_color="green", annotation_text="90th day", annotation_position="top right")

    return fig


@app.callback(
    Output('graph-debt', 'figure'),
    Output('graph-transaction-payment', 'figure'),
    Input('table-timeseries-id', 'children')
)
def update_figure(jsonified_db):
    tmp_db = pd.read_json(jsonified_db, orient='split')

    tmp_db_debt = pd.melt(tmp_db[['dia', 'divida_principal', 'divida_total']],
                          id_vars=['dia'],
                          value_vars=['divida_principal', 'divida_total'])

    tmp_db_payment = pd.melt(tmp_db[['dia', 'transacionado', 'pagamento_diario', 'amortizacao_principal_diario']],
                             id_vars=['dia'],
                             value_vars=['transacionado', 'pagamento_diario', 'amortizacao_principal_diario'])

    return create_time_series(tmp_db_debt, title="Loan Debt"), create_time_series(tmp_db_payment, title="Transactions and Debt Payment")


@app.callback(
    Output('header-info', 'children'),
    Input('table-const-id', 'children')
)
def update_headers(jsonified_db):
    tmp_db = pd.read_json(jsonified_db, orient='split', typ='series')
    loan_contract, _ = tmp_db['desembolso'].split('T')
    loan_overdue, _ = tmp_db['vencimento'].split('T')
    subsegment = tmp_db['subsegmento']
    valor_emprestado = tmp_db['valor_emprestado']
    pgto_diario_esperado = tmp_db['pgto_diario_esperado']
    default = 'Sim' if tmp_db['y'] else 'Nao'
    return [
        html.Div([
            html.H4('Company Sector'),
            html.H3(f'{subsegment}')], className='info-cards'),
        html.Div([
            html.H4('Default'),
            html.H3(f'{default}')], className='info-cards'),
        html.Div([
            html.H4('Loan Value'),
            html.H3(f'{valor_emprestado}')], className='info-cards'),
        html.Div([
            html.H4('Loan Contract'),
            html.H3(f'{loan_contract}')], className='info-cards'),
        html.Div([
            html.H4('Expected Loan Over Due'),
            html.H3(f'{loan_overdue}')], className='info-cards'),
        html.Div([
            html.H4('Expected Loan Daily Payment'),
            html.H3(f'R$ {pgto_diario_esperado:.2f}')], className='info-cards')]


# Run app and display result inline in the notebook
app.run_server(mode='inline', port=8080, debug=True)

In [5]:
dict_loan_timeseries[59438]

,dia,dias_pos_desembolso,divida_total,divida_principal,pagamento_diario,amortizacao_principal_diario,transacionado
0,2021-02-23,0,8802.32,8769.0,0.00,0.0,0.0
1,2021-02-24,1,8819.77,8769.0,0.00,0.0,0.0
2,2021-02-25,2,8814.18,8769.0,23.07,0.0,0.0
3,2021-02-26,3,8809.88,8769.0,21.77,0.0,0.0
4,2021-02-27,4,8827.34,8769.0,0.00,0.0,0.0
5,2021-02-28,5,8844.83,8769.0,0.00,0.0,0.0
